# Introduction to Aether

*This demo is a companion to 0.2 - Introduction to Qiskit*

Qiskit is the largest, most feature rich and most well developed framework for quantum computing. This is obviously a good thing in general, but it can be overwhelming for those getting started.

For that reason, we have created Aether: the smallest, least feature rich and least developed framework for quantum computing! It has everything you need to get to know single and two qubit circuits, and do so with the same syntax as Qiskit. By mastering Aether, you'll be well on your way to mastering Qiskit.

Aether has also been designed to be able to work on microcontroller devices. This includes the PewPew, as well as many other devices used for education and hobbies. It is compatible with MicroPython and CircuitPython, both of which are minimal implementations of Python designed to run on microcontrollers. Many Python packages have minimal versions made to be compatible with these implementations. Aether is similarly the MicroPython-compatible version of Qiskit.

## Basics of Aether (and Qiskit)

### Quantum Circuits

The heart of quantum computing is the circuit. In Qiskit and Aether, our circuit is represented by a Python object, known as the `QuantumCircuit` object. The following line creates such an object, and names it `qc`.

```python
qc = QuantumCircuit(n,n)
```

Here the first `n` is the number of qubits, and the second `n` is the number of output bits. In Qiskit, these can be different numbers in general. However, they are often simply chosen to be equal. Aether restricts us to this standard case. Aether also retricts to `n` being either 1 or 2, as it cannot similate larger numbers of qubits. Many interesting things can be done with just two qubits, so this will not hamper us. However, Qiskit supports any number of qubits.

### Quantum gates

The gates that can be added to a quantum circuit are

```python
qc.x(j)
qc.rx(theta,j)
qc.h(j)
qc.cx(j,k)
```

Here `j` is the number representing the qubit: `0` for a single qubit circuit, or `0` and `1` for a two qubit circuit.

Other gates are also available in Qiskit. However, since everything can be built out of the basic operations we have here, nothing is lost in Aether by restricting to them.

### Measurements

Measurement is the process of extracting an output from a qubit. These outputs take the form of normal bits. Measurement can be performed using the command

```python
qc.measure(j,j)
```

This measures qubit `j` and places the result in output bit `j`. In Qiskit, these two arguments do not neccessarily need to be equal: the labelling of a qubit and the bit used for its output can be different. However, they are often chosen to be the same. Aether only supports this standard case, so the two arguments must be equal.

Measurements in Qiskit can be placed at any point in the circuit. The simulators of Qiskit will quite happily run such quantum programs. However, when running on current prototype quantum hardware, the only supported case is to have all measurements made at the very end. This is also only case supported by Aether, so no gates for a given qubit should be placed after its measurement.

### Executing a Circuit

To simulate a circuit and extract results, the `execute` function is used. This has an argument `shots`, which determines how many times you wish to repeatedly run the circuit to extract statistics. It also has an argument `get` which determines the form in which the results are given.

For example, to get a list of `shots=5` output bit strings, use

```python
m = execute(qc,shots=5,get='memory')
```

The result will look something like `m = ['11','00','01','01','00']`.

To get a dictionary detailing how many of `shots=1024` samples result in each output, use

```python
c = execute(qc,shots=1024,get='counts')
```

The result will look something like `c = {'00':240,'01':275,'10':212,'11':299}`.

To get a state vector describing the state at output, no measurement gates or output bits are required. You are therefore able to initialize the circuit with

```python
qc = QuantumCircuit(n)
```

The state vector is extracted with

```python
state = execute(qc,shots=1024,get='statevector')
```

This syntax for extracting outputs in Aether is slightly different to that in Qiskit. This is due to Aether not needing to be compatible with as many different use cases as Qiskit, and because running Aether on the microcontrollers means that there is not a lot of RAM going spare.

Specifically, the Qiskit equivalents of the execute commands listed above are

```python
m = execute(qc,shots=5).result().get_memory()
c = execute(qc,shots=1024).result().get_counts()
state = execute(qc,shots=1024).result().get_statevector()
```

Note also that the format for complex numbers in the statevector is different in Aether. A complex number of the form $a + i b$ is represented by the list `[a,b]` in Aether, rather than `a + bj` as in Qiskit. A statevector composed only of real numbers can be initialized with a list of real numbers as normal.

## Using Aether

We will now use our quantum tools on the PewPew. To do this, take the file named 'Aether.py', included in this folder, and place it on your PewPew. You will then be able to run the program below. If you are using the emulator, the 'Aether.py' file is already where it needs to be.

The following program shows a simple example of using Aether. It is largely the same as the program in the last section. The main difference is that it no longer checks for user input inside the loop, but instead executes a two qubit quantum program designed to randomly generate random outputs (details regarding how this works will follow in later workshops).

```python
qc = QuantumCircuit(2,2) # create an empty circuit with two qubits and two output bits
# put a hadamard on both qubits
qc.h(0) 
qc.h(1)
# put a measurement on both qubit, whose results go to the corresponding output bit
qc.measure(0,0)
qc.measure(1,1)
```

The results are extracted using `shots=1` and `get='memory'`.

```python
m = execute(qc,shots=1,get='memory')
```

The result contained in `m` is a list containing a single result. This will one of the four possible two-bit outputs: `'00'`, `'01'`, `'10'` and `'11'`. This is accessed by looking at the first (and only) element of the list `m` using `m[0]`.

These four outputs can be interpreted as the binary representations of the numbers 0, 1 , 2 and 3. Since these are exactly the allowed values for the brightness on a PewPew, let's convert them to an integer using `int(m[0],2)` and use them as the brightness of pixel (1,2). This replaces the user input from the previous program.

In [ ]:
%matplotlib notebook

In [ ]:
import pew # setting up tools for the pewpew
from aether import QuantumCircuit, execute # setting up tools for quantum

pew.init() # initialize the game engine...
screen = pew.Pix() # ...and the screen

qc = QuantumCircuit(2,2) # create an empty circuit with two qubits and two output bits
# put a hadamard on both qubits
qc.h(0) 
qc.h(1)
# put a measurement on both qubit, whose results go to the corresponding output bit
qc.measure(0,0)
qc.measure(1,1)

# loop over the square centered on (1,2) and make all dim
(X,Y) = (1,2)
for dX in [+1,0,-1]:
    for dY in [+1,0,-1]:
        screen.pixel(X+dX,Y+dY,2)
        
screen.pixel(X,Y,0) # turn off pixel at (1,2) 

while True: # loop which checks for user input and responds

    # execute the circuit and get a single sample of memory
    m = execute(qc,shots=1,get='memory')
    
    # the resulting bit string m[0] is converted to the corresponding number and used as brightness
    screen.pixel(X,Y,int(m[0],2))

    pew.show(screen) # update screen to display the above changes

    pew.tick(1/6) # pause for a sixth of a second

Another way to display the outputs `'00'`, `'01'`, `'10'` and `'11'` is to use two squares: one for each bit value. We can turn the pixel at (1,2) on and off to show whether the left bit is `1` or `0`, and do the same for the right bit and pixel (6,2). The left bit value is accessed using `m[0][0]`, and the right bit value using `m[0][1]`.

In [ ]:
import pew # setting up tools for the pewpew
from aether import QuantumCircuit, execute # setting up tools for quantum

pew.init() # initialize the game engine...
screen = pew.Pix() # ...and the screen

qc = QuantumCircuit(2,2) # create an empty circuit with two qubits and two output bits
# put a hadamard on both qubits
qc.h(0) 
qc.h(1)
# put a measurement on both qubit, whose results go to the corresponding output bit
qc.measure(0,0)
qc.measure(1,1)

# loop over the squares centered on (1,2) and (6,2) and make all dim
for (X,Y) in [(1,2),(6,2)]:
    for dX in [+1,0,-1]:
        for dY in [+1,0,-1]:
            screen.pixel(X+dX,Y+dY,2)
        
for (X,Y) in [(1,2),(6,2)]:
    screen.pixel(X,Y,0) # turn off the center pixels of the squares  

while True: # loop which checks for user input and responds

    # execute the circuit and get a single sample of memory
    m = execute(qc,shots=1,get='memory')
    
    # turn the pixel (1,2) on or off depending on whether the first bit value is 1 or 0
    if m[0][0]=='1':
        screen.pixel(1,2,3)
    else:
        screen.pixel(1,2,0)
    # do the same for pixel (6,2)
    if m[0][1]=='1':
        screen.pixel(6,2,3)
    else:
        screen.pixel(6,2,0)

    pew.show(screen) # update screen to display the above changes

    pew.tick(1/6) # pause for a sixth of a second

These programs were a simple 'Hello World', just to show that Aether is working. Now let's do something with it.

**[Click here for the next notebook](W.1.1.ipynb)**